In [30]:
import pandas as pd
import numpy as np

In [31]:
crime_sort = pd.read_hdf('../data/crime.h5', 'crime') \
.set_index('REPORTED_DATE') \
.sort_index()

In [32]:
crime_sort.head()

,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,GEO_LON,GEO_LAT,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
REPORTED_DATE,,,,,,,
2012-01-02 00:06:00,aggravated-assault,aggravated-assault,-104.816860,39.796717,montbello,1,0
2012-01-02 00:06:00,violation-of-restraining-order,all-other-crimes,-104.816860,39.796717,montbello,1,0
2012-01-02 00:16:00,traffic-accident-dui-duid,traffic-accident,-104.971851,39.736874,cheesman-park,0,1
2012-01-02 00:47:00,traffic-accident,traffic-accident,-104.951824,39.740168,congress-park,0,1
2012-01-02 01:35:00,aggravated-assault,aggravated-assault,-104.998131,39.749922,union-station,1,0


In [33]:
common_attrs = set(dir(crime_sort.index)) & \
 set(dir(pd.Timestamp))
print([attr for attr in common_attrs if attr[0] != '_'])

['days_in_month', 'is_year_end', 'max', 'nanosecond', 'is_quarter_start', 'is_month_end', 'is_leap_year', 'tz_localize', 'year', 'timetz', 'is_year_start', 'daysinmonth', 'microsecond', 'month_name', 'week', 'day_name', 'normalize', 'dayofweek', 'date', 'tz_convert', 'is_quarter_end', 'to_numpy', 'quarter', 'to_period', 'minute', 'time', 'min', 'weekofyear', 'resolution', 'freq', 'round', 'strftime', 'to_julian_date', 'second', 'tzinfo', 'hour', 'floor', 'weekday', 'day', 'is_month_start', 'dayofyear', 'tz', 'to_pydatetime', 'freqstr', 'month', 'ceil']


In [34]:
crime_sort.index.day_name().value_counts()

Monday       70024
Friday       69621
Wednesday    69538
Thursday     69287
Tuesday      68394
Saturday     58834
Sunday       55213
Name: REPORTED_DATE, dtype: int64

In [35]:
crime_sort.groupby(lambda x: x.day_name()) \
 ['IS_CRIME', 'IS_TRAFFIC'].sum()

<ipython-input-35-e3383ea4ec48>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  crime_sort.groupby(lambda x: x.day_name()) \


,IS_CRIME,IS_TRAFFIC
Friday,48833,20814
Monday,52158,17895
Saturday,43363,15516
Sunday,42315,12968
Thursday,49470,19845
Tuesday,49658,18755
Wednesday,50054,19508


In [36]:
funcs = [lambda x: x.round('2h').hour, lambda x: x.year]

In [37]:
funcs

[<function __main__.<lambda>(x)>, <function __main__.<lambda>(x)>]

In [38]:
cr_group = crime_sort.groupby(funcs) \
 ['IS_CRIME', 'IS_TRAFFIC'].sum()

<ipython-input-38-3b9e47df6888>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cr_group = crime_sort.groupby(funcs) \


In [39]:
cr_group.head()

IS_CRIME  IS_TRAFFIC
0 2012      2422         919
  2013      4040         792
  2014      5649         978
  2015      5649        1136
  2016      5377         980

In [40]:
cr_final = cr_group.unstack()

In [41]:
cr_final.head()

IS_CRIME                               IS_TRAFFIC                          \
      2012  2013  2014  2015  2016  2017       2012  2013  2014  2015  2016   
0     2422  4040  5649  5649  5377  3811        919   792   978  1136   980   
2     1888  3214  4245  4050  4091  3041        718   652   779   773   718   
4     1472  2181  2956  2959  3044  2255        399   378   424   471   464   
6     1067  1365  1750  2167  2108  1567        411   399   479   494   593   
8     2998  3445  3727  4161  4488  3251       1957  1955  2210  2331  2372   

         
   2017  
0   782  
2   537  
4   313  
6   462  
8  1828

In [42]:
cr_final.style.highlight_max(color='blue')

In [43]:
cr_final.stack()

IS_CRIME  IS_TRAFFIC
0  2012      2422         919
   2013      4040         792
   2014      5649         978
   2015      5649        1136
   2016      5377         980
...           ...         ...
22 2013      4318        1330
   2014      5496        1532
   2015      5626        1671
   2016      5637        1472
   2017      4358        1072

[72 rows x 2 columns]

In [44]:
cr_final.xs('IS_TRAFFIC', axis='columns', level=0).head()

,2012,2013,2014,2015,2016,2017
0,919,792,978,1136,980,782
2,718,652,779,773,718,537
4,399,378,424,471,464,313
6,411,399,479,494,593,462
8,1957,1955,2210,2331,2372,1828


# Cross Section

In [45]:
cr_final.xs(2016, axis='columns', level=1).head()

,IS_CRIME,IS_TRAFFIC
0,5377,980
2,4091,718
4,3044,464
6,2108,593
8,4488,2372
